In [14]:
import math
import numpy as np
import pandas as pd

#Ler conjunto de dados de preenchimento normal
normal = open("./normal/normal.csv").read().splitlines()

# conjunto de ataques
bufferoverflow = open('./ataques/bufferoverflow.csv').read().splitlines()
crlfi = open('./ataques/crlfi.csv').read().splitlines()
formatstring = open('./ataques/formatstring.csv').read().splitlines()
ldapi = open('./ataques/ldapi.csv').read().splitlines()
sqli = open('./ataques/sqli.csv').read().splitlines()
ssi = open('./ataques/ssi.csv').read().splitlines()
xpath = open('./ataques/xpath.csv').read().splitlines()
xss = open('./ataques/xss.csv').read().splitlines()

print('Quantidades de dados normais')
print('normal=', len(normal))
print('\nQuantidades de ataques')
print('bufferoverflow=', len(bufferoverflow))
print('crlfi=', len(crlfi))
print('formatstring=', len(formatstring))
print('ldapi=', len(ldapi))
print('sqli=', len(sqli))
print('xpath=', len(xpath))
print('xss=', len(xss))

Quantidades de dados normais
normal= 8161

Quantidades de ataques
bufferoverflow= 412
crlfi= 327
formatstring= 41
ldapi= 74
sqli= 43013
xpath= 175
xss= 4817


In [16]:
from bitarray import bitarray
import mmh3

class BloomFilter:
    
    def __init__(self, size, hash_count):
        self.size = size
        self.hash_count = hash_count
        self.bit_array = bitarray(size)
        self.bit_array.setall(0)
        
    def add(self, string):
        for seed in range(self.hash_count):
            result = mmh3.hash(string, seed) % self.size
            self.bit_array[result] = 1
            
    def lookup(self, string):
        for seed in range(self.hash_count):
            result = mmh3.hash(string, seed) % self.size
            if self.bit_array[result] == 0:
                return False # Se 0 é pq não está no array
        return True # Se 1 é pq talvez está no array

In [17]:
# Escolhe a base de treino
treino = normal

n = len(treino)
p = 0.01

#Calcular tamanho do Filtro
m = -(n*math.log(p))/(math.log(2)**2)
k = (m/n) * math.log(2)

# Arredonda o valores de m e k
m_tam = round(m)
k_hash = round(k)

print('Imprime a configuração ideal do Filtro para a quantidade de itens da base de treino')
print("Qtde de itens na base de treino:", n)
print("Probabilidade de FP:", p)
print("Tamanho do Filtro:", m_tam)
print("Nº de funções hash:", k_hash)

Imprime a configuração ideal do Filtro para a quantidade de itens da base de treino
Qtde de itens na base de treino: 8161
Probabilidade de FP: 0.01
Tamanho do Filtro: 78224
Nº de funções hash: 7


In [18]:
def consultaFiltro(perfis):
        result = 0
        for i in perfis:
            if bf.lookup(i) == True:
                result += 1
        return result

In [50]:
# inicia o filtro com os parametros padrões
bf = BloomFilter(200000, k_hash)
resultado = []

# preenche o filtro com os dados de treino
for i in treino:
    bf.add(i)
    
resultado.append(consultaFiltro(bufferoverflow))
resultado.append(consultaFiltro(crlfi))
resultado.append(consultaFiltro(formatstring))
resultado.append(consultaFiltro(ldapi))
resultado.append(consultaFiltro(sqli))
resultado.append(consultaFiltro(xpath))
resultado.append(consultaFiltro(xss))

#df = pd.DataFrame(np.array(resultado).reshape(7,8), columns = list('Filtrou'), )
#df
resultado

[1, 0, 0, 0, 2, 0, 0]